In [ ]:
!pip install -q streamlit


In [ ]:
%pwd

'/content'

In [ ]:
%%writefile app.py
import streamlit as st

from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from plotly import graph_objs as go

from darts import TimeSeries
from darts.models import RNNModel
from darts.metrics import mape
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.models import forecasting
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller

import yfinance as yf


#Getting date from one year ago
one_year_ago = datetime.now() - relativedelta(years=1) 
one_year_ago = one_year_ago.strftime("%Y-%m-%d")

#Getting date today
today = datetime.now()
today = today.strftime("%Y-%m-%d")

#Adding one day to the date one year ago 
one_year_ago_plus_one = datetime.now() - relativedelta(years=1) + timedelta(days=1)
one_year_ago_plus_one = one_year_ago_plus_one.strftime("%Y-%m-%d")


st.title('Stock Forecast App')

stocks = ('AAPL', 'ACN', 'ADBE', 'ADI', 'ADSK', 'AKAM', 'AMAT', 'AMD', 'ANET', 'ANSS', 'AVGO', 'CDNS', 'CDW', 'CRM', 'CSCO', 'CTSH', 'DXC', 'ENPH', 'EPAM', 'FFIV', 'FICO', 'FSLR', 'FTNT', 'GLW', 'INTU', 'IT', 'JNPR', 'KEYS', 'KLAC', 'LRCX', 'MCHP', 'MPWR', 'MSI', 'MU', 'NOW', 'NTAP', 'NVDA', 'NXPI', 'ON', 'PTC', 'QCOM', 'QRVO', 'ROP', 'ROP', 'SEDG', 'SNPS', 'SWKS', 'TEL', 'TER', 'TRMB', 'TXN', 'TYL', 'VRSN', 'ZBRA')
selected_stock = st.selectbox('Select the stock you wish to view/predict', stocks)

def load_data_and_process(tick):
    #Accessing YFinance
    ticker= yf.Ticker("fico")

    #Getting pandas dataframe of stock data from one year ago
    df = ticker.history(start=one_year_ago, end=today, interval="1d")

    idx = pd.date_range(start=one_year_ago_plus_one, end=today)
    #Making index into Datetime index
    df.index = pd.DatetimeIndex(df.index)
    df.index = df.index.tz_localize(None) 

    #Reindexing dataframe to fill in missing dates due to stock market closing on weekends
    df = df.reindex(idx, method = 'pad')

    return df
  
def data_transform(df):
    #Put dataframe into a Darts "Timeseries" object so that data can be fed into a Darts forecasting model. For more information on Timeseries objects, please look here: https://unit8co.github.io/darts/generated_api/darts.timeseries.html 
    series = TimeSeries.from_dataframe(df)

    #Drop all columns besides "Close" Column
    adj_series = series.drop_columns(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'])

    #Transform training set, validation set, and entire series to values between 0 and 1.
    transformer = Scaler()

    #train_transformed = transformer.fit_transform(training)
    #val_transformed = transformer.transform(validation)

    series_transformed = transformer.transform(adj_series)

    return series_transformed

def LSTM_Model(series_transformed, days):
    my_model = RNNModel(
    model="LSTM",
    n_rnn_layers = 3, # Number of LSTM layers
    hidden_dim=4,
    dropout=0.2,
    batch_size=16,
    n_epochs=10,
    optimizer_kwargs={"lr": 1e-3}, #learning rate
    model_name="Air_RNN",
    log_tensorboard=True,
    random_state=42,
    training_length=20,
    input_chunk_length=14,
    force_reset=True,
    save_checkpoints=True,
    )

    #Fit model on training data
    my_model.fit(series_transformed)

    pred = my_model.predict(n=days)
    #Perform inverse scaling on prediction
    predicted_values = transformer.inverse_transform(pred)
    return predicted_values

def plot_predictions(predicted_values, adj_series):
    fig = plt.plot(predicted_values)
    fig1 = plt.plot(adj_series)
    st.pyplot(fig1, fig)


def inverse_transform_data(data):
    original_series = transformer.inverse_transform(data)
    return original_series

#might not need this function
#def forecast_values(days):
    #pred = my_model.predict(n=days)
    ##Perform inverse scaling on prediction
    #predicted_values = transformer.inverse_transform(pred)
    #return predicted_values

#def plot_predictions(predicted_values, adj_series):
    #Plot total data vs prediction
    #adj_series.plot(label='actual')
    #predicted_values.plot(label='forecast')
    #plt.legend();
    #plt.title("Total Data vs Inverse Scaled Prediction")

#Likely will not need this
#def train_validation_split(adj_series):
    #Divide dataset into training and validation/testing sets. "Split_after" function is a method of a Darts Timeseries object. https://unit8co.github.io/darts/generated_api/darts.timeseries.html#darts.timeseries.TimeSeries.split_after
    #training, validation = adj_series.split_after(0.8)

#def load_data(ticker):
    #data = yf.download(ticker, START, TODAY)
    #data.reset_index(inplace=True)
    #return data

#def transform_data(ticker):
    #aapl= yf.Ticker(ticker)
    #df = aapl.history(start="2022-04-01", end="2023-04-01", interval="1d")
    #idx = pd.date_range(start='2022-04-01', end='2023-04-01')
    #df.index = pd.DatetimeIndex(df.index)
    #df.index = df.index.tz_localize(None) 
    #df = df.reindex(idx, method = 'pad')
    #series = TimeSeries.from_dataframe(df)
    #adj_series = series.drop_columns(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'])
    #return adj_series

data_load_state = st.text('Loading data...')
data = load_data_and_process(selected_stock)
data_load_state.text('Loading data... done!')
st.subheader('Original Data From Previous Year')
st.write(data.tail())


transform_data_state = st.text('Transforming data...')
transformed_data = data_transform(data)
transform_data_state.text('Transforming data...done')

predicted_values = LSTM_Model(transformed_data,60)

original_series = inverse_transform_data(transformed_data)
plot_predictions(predicted_values, original_series)


Overwriting app.py


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
#import shutil
from sklearn.preprocessing import MinMaxScaler
#from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape, mse
#from darts.utils.statistics import check_seasonality, plot_acf
#from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.models import forecasting
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller

import yfinance as yf
aapl= yf.Ticker(selected_stock)
df = aapl.history(start="2022-04-01", end="2023-04-01", interval="1d")

# data.reset_index(inplace=True)
# data = data[['Date', 'Close']]

idx = pd.date_range(start='2022-04-01', end='2023-04-01')
df.index = pd.DatetimeIndex(df.index)
df.index = df.index.tz_localize(None) 
# # #df['Date'] = pd.to_datetime(df['Date'])
# # #print(df)
# # #df.set_index='Date'
# # #print(df)
# # df.index = pd.DatetimeIndex(df.index)
# # print(df)
df = df.reindex(idx, method = 'pad')
print(df)

NameError: ignored

In [ ]:
series = TimeSeries.from_dataframe(df)

adj_series = series.drop_columns(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'])
training, validation = adj_series.split_after(0.8)

from darts.dataprocessing.transformers import Scaler
transformer = Scaler()
train_transformed = transformer.fit_transform(training)
val_transformed = transformer.transform(validation)
series_transformed = transformer.transform(adj_series)



In [ ]:
my_model = RNNModel(
    model="LSTM",
    n_rnn_layers = 3,
    hidden_dim=4,
    dropout=0,
    batch_size=16,
    n_epochs=10,
    optimizer_kwargs={"lr": 1e-3},
    model_name="Air_RNN",
    log_tensorboard=True,
    random_state=42,
    training_length=20,
    input_chunk_length=3,
    force_reset=True,
    save_checkpoints=True,
)
my_model.fit(train_transformed)

In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 1.079s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.461s
your url is: https://tricky-rats-search.loca.lt
^C
